#### 상용자 정의 모델 클래스
- 부모클래스 : nn.Module
- 필수오버라이딩 
    * _ _init_ _() :  모델 층 구성 즉, 설계
    * forward() : 순방향 학습 진행 코드 구현

In [1]:
## 모듈로딩
import torch                                # 텐서 관련 모듈 
import torch.nn as nn                       # 인공신경망 관련 모듈
import torch.nn.functional as F             # 인공신경망 관련 함수들 모듈 ( 손실함수, 활성화함수 등등 )
import torch.optim  as optim                # 최적화 관련 모듈 (가중치, 절편 빠르게 찾아주는 알고리즘)
from torchinfo import summary               # 모델 구조 및 정보 관련 모듈 
from torchmetrics.regression import *       # 회귀 성능 지표 관련 모듈
from torchmetrics.classification import *   # 분류 성능 지표 관련 모듈


In [2]:
# 랜덤 고정
torch.manual_seed(1)

# 텐서 저장 및 실행 위치 설정
DEVICE='cuda' if torch.cuda.is_available() else 'cpu'

- [기본] 신경망클래스 <hr>
    * 입력층 - 입력 : 피쳐수 고정
    * 출력층 - 출력 : 타겟수 고정
    * 은닉층 - 고정

In [3]:
# 모델 설계
# 데이터셋 : 피쳐 4개,  타겟 1개,   회귀
# 입 력 층 : 입력    4개     출력   20개      AF  ReLU
# 은 닉 층 : 입력   20개     출력  100개      AF  ReLU
# 출 력 층 : 입력  100개     출력    1개      AF  X , Sigmoid & Softmax

In [4]:
class MyModel(nn.Module):
    
    # 인스턴스/객체 생성 시 자동호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    def __init__(self):
        # 부모클래스 생성
        super().__init__()
        # 자식클래스의 인스턴스 속성 설정
        self.input_layer= nn.Linear(4, 20)    # W 4 + b 1 => 1P , 5 * 20 = 100개 변수
        self.hidden_layer=nn.Linear(20, 100)  # W 20 + b 1 => 1P , 21 * 100  = 2100개 변수
        self.output_layer=nn.Linear(100, 1)   # W 100 + b 1 => 1P, 101 * 1 = 101개 변수
        
        
    # 순방향/전방향 학습 진행 시 자동호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋
    def forward(self, x):
        print('calling forward()')
        y=self.input_layer(x)           # 1개 퍼셉트론 : y=x1W1+x2W2+x3W3+x4W4+b
        y=F.relu(y)                     # 0 <= y   ----> 죽은 relu ==> leakyReLu
        
        y=self.hidden_layer(y)          # 1개 퍼셉트론 : y=x1W1+x2W2+~~~+x20W20+b
        y=F.relu(y)
        
        return self.output_layer(y)     # 1개 퍼셉트론 : y=x1W1+x2W2+~~~+x100W100+b
        

In [5]:
# 입력 피쳐 수가 동적인 모델 
class MyModel2(nn.Module):
    
    # 인스턴스/객체 생성 시 자동호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    def __init__(self, in_features):
        # 부모클래스 생성
        super().__init__()
        # 자식클래스의 인스턴스 속성 설정
        self.input_layer= nn.Linear(in_features, 20)    # W 4 + b 1 => 1P , 5 * 20 = 100개 변수
        self.hidden_layer=nn.Linear(20, 100)            # W 20 + b 1 => 1P , 21 * 100  = 2100개 변수
        self.output_layer=nn.Linear(100, 1)             # W 100 + b 1 => 1P, 101 * 1 = 101개 변수
        
        
    # 순방향/전방향 학습 진행 시 자동호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋
    def forward(self, x):
        print('calling forward()')
        y=self.input_layer(x)                           # 1개 퍼셉트론 : y=x1W1+x2W2+x3W3+x4W4+b
        y=F.relu(y)                                     # 0 <= y   ----> 죽은 relu ==> leakyReLu
        
        y=self.hidden_layer(y)                          # 1개 퍼셉트론 : y=x1W1+x2W2+~~~+x20W20+b
        y=F.relu(y)
        
        return self.output_layer(y)                     # 1개 퍼셉트론 : y=x1W1+x2W2+~~~+x100W100+b
    

In [6]:
# 입력 피쳐 수, 은닉층 퍼셉트론 수 동적인 모델 
class MyModel3(nn.Module):
    
    # 인스턴스/객체 생성 시 자동호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    def __init__(self, in_features, in_out, h_out):
        # 부모클래스 생성
        super().__init__()
        # 자식클래스의 인스턴스 속성 설정
        self.input_layer= nn.Linear(in_features, in_out)    # W 4 + b 1 => 1P , 5 * 20 = 100개 변수
        self.hidden_layer=nn.Linear(in_out, h_out)          # W 20 + b 1 => 1P , 21 * 100  = 2100개 변수
        self.output_layer=nn.Linear(h_out, 1)               # W 100 + b 1 => 1P, 101 * 1 = 101개 변수
        
        
    # 순방향/전방향 학습 진행 시 자동호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋
    def forward(self, x):
        print('calling forward()')
        y=self.input_layer(x)                               # 1개 퍼셉트론 : y=x1W1+x2W2+x3W3+x4W4+b
        y=F.relu(y)                                         # 0 <= y   ----> 죽은 relu ==> leakyReLu
        
        y=self.hidden_layer(y)                              # 1개 퍼셉트론 : y=x1W1+x2W2+~~~+x20W20+b
        y=F.relu(y)
        
        return self.output_layer(y)                         # 1개 퍼셉트론 : y=x1W1+x2W2+~~~+x100W100+b
        

In [7]:
# 은닉층의 개수가 동적인 모델
class MyModel4(nn.Module):
    
    # 인스턴스/객체 생성 시 자동호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    def __init__(self, in_feature, out_feature, in_hiddend, hidden_cnt):
        # 부모클래스 생성
        super().__init__()
        
        # 자식클래스의 인스턴스 속성 설정
        self.input_layer= nn.Linear(in_feature, in_hiddend) 
        self.h1_layer=nn.ModuleList([nn.Linear(in_hiddend, in_hiddend) for _ in range(hidden_cnt)])
        self.output_layer=nn.Linear(in_hiddend, out_feature)
        

In [8]:
m4=MyModel4(5, 1, 10, 3)
print(m4)
for name, param in m4.named_parameters():
    print(name, param.shape)

MyModel4(
  (input_layer): Linear(in_features=5, out_features=10, bias=True)
  (h1_layer): ModuleList(
    (0-2): 3 x Linear(in_features=10, out_features=10, bias=True)
  )
  (output_layer): Linear(in_features=10, out_features=1, bias=True)
)
input_layer.weight torch.Size([10, 5])
input_layer.bias torch.Size([10])
h1_layer.0.weight torch.Size([10, 10])
h1_layer.0.bias torch.Size([10])
h1_layer.1.weight torch.Size([10, 10])
h1_layer.1.bias torch.Size([10])
h1_layer.2.weight torch.Size([10, 10])
h1_layer.2.bias torch.Size([10])
output_layer.weight torch.Size([1, 10])
output_layer.bias torch.Size([1])


In [9]:
# 은닉층의 개수가 동적인 모델
class MyModel5(nn.Module):
    
    # 인스턴스/객체 생성 시 자동호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    def __init__(self, in_in, in_out, out_out, h_ins=[], h_outs=[]):
        # 부모클래스 생성
        super().__init__()
        # 자식클래스의 인스턴스 속성 설정
        self.input_layer= nn.Linear(in_in, h_ins[0] if len(h_ins) else in_out) 
        
        self.h1_layer=nn.ModuleList()
        for idx in range(len(h_ins)):
            self.h1_layer.append(nn.Linear(h_ins[idx], h_outs[idx]))
        self.output_layer=nn.Linear(h_outs[-1] if len(h_outs) else in_out, out_out)
        
    def forward(self, x):
        y=F.relu(self.input_layer(x))
        for linear in self.h1_layer:
            y=F.relu(linear(y))
        return self.output_layer(y)

In [10]:
#h_ins, h_outs =[10,30,50], [30,50,70]
h_ins, h_outs =[10], [30]
m5=MyModel5(3,10,1,h_ins, h_outs)
print(m5)
summary(m5)
for name, param in m5.named_parameters():
    print(name, param.shape)

MyModel5(
  (input_layer): Linear(in_features=3, out_features=10, bias=True)
  (h1_layer): ModuleList(
    (0): Linear(in_features=10, out_features=30, bias=True)
  )
  (output_layer): Linear(in_features=30, out_features=1, bias=True)
)
input_layer.weight torch.Size([10, 3])
input_layer.bias torch.Size([10])
h1_layer.0.weight torch.Size([30, 10])
h1_layer.0.bias torch.Size([30])
output_layer.weight torch.Size([1, 30])
output_layer.bias torch.Size([1])


In [11]:
# 학습 진행 ==> 모델인스턴스명(데이터)
# 임의의 데이터 
dataTS=torch.FloatTensor([[1,3,5], [2,4,6]])
targetTS=torch.FloatTensor([[4],[5]])

# 학습
pre_y=m5(dataTS)

print(pre_y)

tensor([[0.1055],
        [0.1103]], grad_fn=<AddmmBackward0>)


In [12]:
m5=MyModel5(5,10,1)
# print(m5)
# summary(m5)

dataTS=torch.FloatTensor([[1,3,5,7,9], [2,4,6, 8,10]])
targetTS=torch.FloatTensor([[4],[5]])
# 학습
pre_y=m5(dataTS)

print(pre_y)

tensor([[0.4418],
        [0.5492]], grad_fn=<AddmmBackward0>)


In [13]:
m4=MyModel4(5, 1, 10, 3)
print(m4)
for name, param in m4.named_parameters():
    print(name, param.shape)

MyModel4(
  (input_layer): Linear(in_features=5, out_features=10, bias=True)
  (h1_layer): ModuleList(
    (0-2): 3 x Linear(in_features=10, out_features=10, bias=True)
  )
  (output_layer): Linear(in_features=10, out_features=1, bias=True)
)
input_layer.weight torch.Size([10, 5])
input_layer.bias torch.Size([10])
h1_layer.0.weight torch.Size([10, 10])
h1_layer.0.bias torch.Size([10])
h1_layer.1.weight torch.Size([10, 10])
h1_layer.1.bias torch.Size([10])
h1_layer.2.weight torch.Size([10, 10])
h1_layer.2.bias torch.Size([10])
output_layer.weight torch.Size([1, 10])
output_layer.bias torch.Size([1])
